# Notebook 13.2: Intro to RAD-seq

### Learning objectives: 

By the end of this notebook you will:

1. Understand how restriction digestion can be used to subsample genomes. 
2. Have a general understanding of how genomic libraries are prepared for sequencing.

### Recommended readings:

1. Andrews, Kimberly R., Jeffrey M. Good, Michael R. Miller, Gordon Luikart, and Paul A. Hohenlohe. 2016. “Harnessing the Power of RADseq for Ecological and Evolutionary Genomics.” Nature Reviews Genetics 17 (2): 81. https://doi.org/10.1038/nrg.2015.28.

In [1]:
import numpy as np

### What is RAD-seq?

*Restriction-site associated DNA (RAD) sequencing* is a method for subsampling genomes to concentrate sequenced reads onto particular portions of the genome rather than being spread randomly across the entire genome. In contrast to 'whole-genome sequencing' (WGS) approaches RAD-seq is considered a 'reduced-representation' sequencing method. 

Despite this term, the RAD methodology itself is not a different form of *sequencing* than we've seen before -- most RAD-seq libraries are sequenced on Illumina or similar devices to produce short reads -- rather, RAD-seq is a protocol for preparing your DNA before you put it on the sequencing machine such that only a subset of regions of the genome will be sequenced. 

### Why RAD-seq?
There are many types of questions we can investigate using genomics that do not require sequencing the entire genome. Most notably, this includes most forms of *comparative genomics* where we are interested in the *differences* between individuals, populations or species. 

As we've discussed before, if you want to know about sequence differences at an individual gene among a number of different species then the most efficient way to answer this is not to sequence their entire genomes. Instead, it is much cheaper to use Sanger sequencing, or to design probes to target just that particular region of the genome (a topic we'll return to later).

### Genomic library preparation
Genomic library preparation for Illumina sequencing involves combining oligonucleotide molecules in solution, in a particular order, to build larger molecules that will bind by complementarity to the sequencing machine where they can be read using "sequencing by synthesis". Below we will build oligonucleotides bioinformatically for a RAD-seq library following the Original RAD method (*sensu* Andrews et al.). Our library preparation has two distinct goals:

+ digest the DNA with the `PstI` restriction enzyme (CTGCA^G) 
+ ligate unique six-base barcodes (e.g., AACCTT) to the overhang.
+ ligate Illumina adapters to the molecule.  


In [2]:
# The TruSeq Illumina adapters
ILLUMINA_ADAPTER_1 = "AGATCGGAAGAGCACACGTCTGAACTCCAGTCA"
ILLUMINA_ADAPTER_2 = "AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT"

### Let's simulate the single-digest original RAD-seq method. 

We will use code to bioinformatically follow the molecular protocol described in Figure 1 by Andrews et al. for how a RAD-seq library is prepared in the "Original RAD" method. 

In [3]:
def random_sequence(N):
    "returns a randomly generated DNA scaffold of length N as a string"
    return "".join(np.random.choice(list("ACGT"), size=N))


def restriction_digest(sequence, recognition, cut):
    """
    restriction digest a genome sequence at the given (recognition) site and
    split the site at the given position (cut) to leave overhangs. Returns one
    strand of the resulting fragment: e.g., for recognition=CTGCAG, cut=5 this
    returns the 5' to 3' strand below.
       5' ----G[seq]CTGCA 3'
       3' ACGTC[seq]G---- 5'
    """
    # cut sequence at every occurence of recognition site
    fragments = sequence.split(recognition)
    
    # add overhang that results from sequence splitting within the recognition site
    fragments = [recognition[cut:] + i + recognition[:cut] for i in fragments]
    return fragments
    
    
def complement(sequence):
    "return the complement of a sequence"
    return (sequence
              .replace("C", "g")
              .replace("G", "c")
              .replace("T", "a")
              .replace("A", "t")
              .upper())


def ligate_barcoded_adapters(fragments, recognition, cut, barcode):
    """
    Ligates a barcoded Illumina adapter to each fragment if it has a cleaved
    sequence overhang matching the recognition site. This will fill the molecule
    so that both ends have full recognition sequence and Illumina adapter.
    """
    ligated = []
    for fragment in fragments:
        
        # adapter has the recognition overhang complement attached to it so
        # it can bind to 5' strand and complement with 3' strand.
        # ----G[seq]CTGCA   <- attaches left side of here   
        # ACGTC[seq]G---- 
        ligate_to_this_strand = "{}{}{}".format(
            complement(ILLUMINA_ADAPTER_1)[::-1], 
            complement(barcode)[::-1],
            recognition[-cut:-(len(recognition) - cut)],   
        )
        
        # attaches to this strand and complements overhang on other strand.
        # ----G[seq]CTGCA   <- attaches here
        # ACGTC[seq]G----   
        ligate_to_other_strand = "{}{}".format(
            barcode, 
            ILLUMINA_ADAPTER_1,
        )
        
        # the full molecule has the fragment in the middle
        molecule = "{}{}{}".format(
            ligate_to_this_strand,
            fragment,
            ligate_to_other_strand,
        )
        ligated.append(molecule)
    return ligated

### Steps 1 and 2:
Here we (1) digest the genome into smaller fragments, and (2) ligate barcoded adapters to the fragments. Barcodes are unique sequences of DNA (e.g., AATTCC or TTAACC). For every individual we can ligate a different barcode to their genome fragments which will allow us to uniquely identify which individual the sequence came from. The Illumina adapter sequence is also added at this step. The Illumina adapter is necessary for the molecule to bind to the Illumina sequencing plate. 

In [4]:
# generate a random genome sequence
scaffold = random_sequence(1000000)

# cut up the genome with a restriction enzyme
fragments = restriction_digest(scaffold, "CTGCAG", 5)

# attach a barcode (AATTCC) and adapter to the overhanging end sequence (CTGCA)
barcoded_fragments = ligate_barcoded_adapters(fragments, "CTGCAG", 5, "AATTCC")

<div class="alert alert-success">
    <b>[1] Question:</b> 
    How many fragments are in the library after the restriction digestion? Calculate it from the fragments or barcoded_fragments lists. If we used a restriction enzyme with a longer recognition sequence would it produce more or fewer fragments?
</div>

In [8]:
# answer here
len(barcoded_fragments)

245

### Let's look at the first fragment with adapters and barcodes ligated
We can select the first fragment in the list of barcoded fragments that we just produced and examine its sequence. This is an oligonucleotide, a molecule composed of multiple small nucleotide sequences that have been ligated together. It is a double stranded DNA molecule and as such one strand is complementary to the other. One strand on either end of the molecule contains the same restriction overhang after the genome is restriction digested, and thus we can ligate the same barcoded adapter to each end. Below we print just one strand to represent the final molecule, and thus on one end we see the reverse complement of the barcoded adapter, and at the other end we see the normal barcoded adapter sequence. In between is the fragmented sequence of the genome. 

In [9]:
# select first fragmented molecule
fragment = barcoded_fragments[0]

# print features of the molecule
print("{:<40}adapter I (revcomp)".format(fragment[:33]))
print("{:<40}barcode (revcomp)".format(fragment[33:39]))
print("{:<40}restriction overhang (revcomp)".format(fragment[39:44]))
print("{}...       5' end of strand".format(fragment[44:74]))
print("...{}       3' end of strand".format(fragment[-74:-44]))
print("{:<40}restriction overhang".format(fragment[-44:-39]))
print("{:<40}barcode".format(fragment[-39:-33]))
print("{:<40}adapter I".format(fragment[-33:]))

TGACTGGAGTTCAGACGTGTGCTCTTCCGATCT       adapter I (revcomp)
GGAATT                                  barcode (revcomp)
TGCAG                                   restriction overhang (revcomp)
GCCAGTATTTGCCCGCCACGAACCTTTTCG...       5' end of strand
...TGTAGAATGGTCGTCTCCTGTCTGCTTTCA       3' end of strand
CTGCA                                   restriction overhang
AATTCC                                  barcode
AGATCGGAAGAGCACACGTCTGAACTCCAGTCA       adapter I


### And the same for another fragment
Here we print the same information for another fragment in the list. You can see that the nucleotide sequence before and after the fragment are the same as in the molecule above, but the sequence of the DNA fragment in the middle is different. 

In [10]:
# select third fragmented molecule
fragment = barcoded_fragments[3]

# print features of the molecule
print("{:<40}adapter I (revcomp)".format(fragment[:33]))
print("{:<40}barcode (revcomp)".format(fragment[33:39]))
print("{:<40}restriction overhang (revcomp)".format(fragment[39:44]))
print("{}...       5' end of strand".format(fragment[44:74]))
print("...{}       3' end of strand".format(fragment[-74:-44]))
print("{:<40}restriction overhang".format(fragment[-44:-39]))
print("{:<40}barcode".format(fragment[-39:-33]))
print("{:<40}adapter I".format(fragment[-33:]))

TGACTGGAGTTCAGACGTGTGCTCTTCCGATCT       adapter I (revcomp)
GGAATT                                  barcode (revcomp)
TGCAG                                   restriction overhang (revcomp)
CTCATGGAATGACTTTCGGTCAGTGCAAGG...       5' end of strand
...GGTCGCATCGGCGAACAACGCGGTCCGATA       3' end of strand
CTGCA                                   restriction overhang
AATTCC                                  barcode
AGATCGGAAGAGCACACGTCTGAACTCCAGTCA       adapter I


### Step3: We can repeat this step for many different individuals 
The next step in the protocol is to *multiplex* samples, which means that we can combine the fragments from many individuals together into a single solution. This is OK to do since we've uniquely barcoded the sequences from each individual and so we can separate them later using bioinformatics based on these sequences. Let's generate fragments for four different individuals by using four different barcodes. You can see that we use a random series of six nucleotides as the 4 different barcodes. Ideally, these should be at least two bases different from each other so that even if sequencing errors occur during sequencing we will still be able to assign the reads to samples accurately. 

In [11]:
# generate a random genome sequence
scaffold = random_sequence(1000000)

# cut up the genome with a restriction enzyme
fragments = restriction_digest(scaffold, "CTGCAG", 5)

# attach a barcode and adapter to fragments from four different individuals
bfrags1 = ligate_barcoded_adapters(fragments, "CTGCAG", 5, "AATTCC")
bfrags2 = ligate_barcoded_adapters(fragments, "CTGCAG", 5, "TTCCGG")
bfrags3 = ligate_barcoded_adapters(fragments, "CTGCAG", 5, "TGCATG")
bfrags4 = ligate_barcoded_adapters(fragments, "CTGCAG", 5, "CCAACC")

# combine all samples into multiplexed collection
all_fragments = bfrags1 + bfrags2 + bfrags3 + bfrags4

### Steps 4 & 5: Size selection
The paper by Andrews et al. describes *size selection*. This refers to steps in the preparation of a genomic library that are required so that the size of fragments that are put onto the sequencing machine are in a proper size range. For Illumina sequencing the fragments should be around 300-1000bp in length. Among the different RADseq library preparation protocols some methods (e.g., GBS and ddRAD) simply select fragments that happen to be in the correct range after restriction digestion. The original RAD method differs by using a *shearing* step, which involves breaking the DNA into smaller pieces. After shearing fragments that have been broken to be near the desired size range are then selected. 

Following on our steps above for preparing an original RAD library bioinformatically, we can emulate the shearing step by bioinformatically breaking the fragments into smaller pieces. Because the fragments have adapters ligated to both ends before they are sheared, the shearing will result in two or more smaller fragments that will have either zero, one or two ligated adapters attached to either end. 

In [12]:
def shear_DNA_to_size_select(fragments, size):
    "randomly shears fragments down to the entered size"
    sheared_fragments = []
    
    # iterate over fragments
    for fragment in fragments:
        
        # get breakpoints for shearing into chunks of size 'size'
        nfrags = max(2, int(len(fragment) / size))
        splits = np.linspace(0, len(fragment), nfrags).astype(int)
        
        # slice out bits to represent sheared fragments
        idx = 0
        for end in splits[1:]:
            fragbit = fragment[idx:end]
            sheared_fragments.append(fragbit)
            idx = end
    return sheared_fragments

In [13]:
# break into 300bp bits and only keep bits 
sheared_fragments = shear_DNA_to_size_select(all_fragments, 300)


<div class="alert alert-success">
    <b>[2] Question:</b> 
    How many fragments are in the library after shearing? What type of sequences do you expect to find at the ends of the fragments after shearing? How is this different from the expectation after the initial restriction digestion? Answer in Markdown below.
</div>

In [14]:
# answer here
len(sheared_fragments)

12308

### Steps 6, 7, 8: Ligate Illumina adapter 2

Ligating the second Illumina adapter is a step that varies widely among different RAD preparation protocols. In the original RAD prep it involves repairing the sheared end 
and attaching a poly-A repeat followed by the second Illumina adapter. The resulting fragments will then have the Illumina adapter 2 present either on zero, one or two ends. 

In [15]:
def end_repair_and_adapter_ligation(fragments):
    """
    Add poly-A to non-adapter end of sheared fragments and attach the 
    Illumina adapter II to this blunt end.
    """
    ligated_fragments = []
    
    # iterate through list
    for frag in fragments:  
        
        # add poly-A tail to 3' of other strand if this fragment has ILLumina 1 at 3' end
        if frag[:33] != complement(ILLUMINA_ADAPTER_1)[::-1]:
            frag = "{}{}{}".format(
                complement(ILLUMINA_ADAPTER_2)[::-1],
                complement("AAAAAAAAAAAAAA"),
                frag,
            )
        
        # add poly-A tail to 3' end if this fragment has ILLumina 1 at 5' end
        if frag[-33:] != ILLUMINA_ADAPTER_1:
            frag = "{}{}{}".format(
                frag,
                "AAAAAAAAAAAAAA",
                ILLUMINA_ADAPTER_2,
            )
            
        # store new end repaired fragment
        ligated_fragments.append(frag)
    return ligated_fragments

In [16]:
# ligate poly-A + Illumina adapter 2 to sheared ends
oligonucleotides = end_repair_and_adapter_ligation(sheared_fragments)

### Step 9: PCR amplification

Polymerase chain reaction (PCR) is a method widely used in molecular biology that takes advantage of the process of DNA replication to preferentially replicate some sequences of DNA more than others, and in doing so to enrich a library for the part that we plan to sequence. The preferential replication of some regions over others is accomplished using *primers* which initiate the DNA replication process. 

In this library we use primers that will amplify any molecule that has **both** the Illumina adapter I and Illumina adapter II at either end. Remember, adapter I is attached to the restriction digested end, and adapter II was ligated to any randomly sheared end. Thus there are many molecules in our library currently that have adapter II at both ends, or sometimes adapter I at both ends. We can effectively remove those by replicating only the molecules that have one sheared end and one digested end (they have both types of adapters). After a few cycles of PCR these molecules will make up almost the entire library. Our function below emulates this by simply removing all oligonucleotides from the library that do not have both adapters. In a real PCR amplification the number of copies of each molecule would increase exponentially with additional cycles. 

In [17]:
def PCR(fragments):
    "Only amplify fragments with both Illumina adapters to enrich library"
    library = []
    for fragment in fragments:
        
        # keep oligos with adapter1...adapter2
        if fragment[:33] == complement(ILLUMINA_ADAPTER_1)[::-1]:
            if fragment[-33:] == ILLUMINA_ADAPTER_2:
                library.append(fragment)
                
        # keep oligos with adapter2...adapter1. Return revcomp'd
        if fragment[:33] == complement(ILLUMINA_ADAPTER_2)[::-1]:
            if fragment[-33:] == ILLUMINA_ADAPTER_1:
                library.append(complement(fragment)[::-1])
                
    return library        

In [18]:
# PCR amplify to select the final library for sequencing 
library = PCR(oligonucleotides)


### The final sequenced molecules

Now that we've PCR amplified our library it contains the final template of molecules that will bind to the Illumina sequencing plate for sequencing. You can see the format of one of these oligonucleotides below. 

In [19]:
# select third fragmented molecule
fragment = library[10]

# print features of the molecule
print("{:<40}adapter I (revcomp)".format(fragment[:33]))
print("{:<40}barcode (revcomp)".format(fragment[33:39]))
print("{:<40}restriction overhang (revcomp)".format(fragment[39:44]))
print("{}...       5' end of strand".format(fragment[44:74]))
print("...{}       3' end of strand".format(fragment[-74:-44]))
print("{:<40}poly-A tail".format(fragment[-44:-34]))
print("{:<40}adapter II".format(fragment[-33:]))

TGACTGGAGTTCAGACGTGTGCTCTTCCGATCT       adapter I (revcomp)
GGAATT                                  barcode (revcomp)
TGCAG                                   restriction overhang (revcomp)
GCGCGTTTATCGCCACTATAATCAGGTGTA...       5' end of strand
...TTGCGGCGTGTCTAAGCGGATGGATAGAAA       3' end of strand
AAAAAAAAAA                              poly-A tail
AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT       adapter II


### Similar and related protocols
The steps involved in a RAD-seq library preparation are in fact common to many types of genomic library preparations. Often different variants of a method exist that differ in the types of equipment that are required, and/or in the cost of the reagents that are involved. For example, the cost of a machine to *shear* DNA is quite expensive, as are the adapters that can bind to poly-A labeled sheared ends. An alternative RAD-seq approach called ddRAD has become popular in recent years that avoids this step by instead using two digestion enzymes such that both ends of a DNA fragment can use ligation to attach adapters. 

### tagmentation
We will learn more later about another method for fragmenting genomic DNA called *tagmentation*. This approach uses modified transposomes that have incorporated adapter sequences and insert themselves into a DNA sample to create double-stranded breaks that leave adapters as overhanging ends. Similar to RAD-seq this overhanging end leaves a target that can be used to ligate additional barcodes or tags. We will revisit this method when we return to nanopore sequencing prep methods. 

<div class="alert alert-success">
    <b>[4] Question:</b> 
   
How can a RAD-seq protocol be modified to sample more densely throughout a genome?

</div>

In [18]:
# answer here

<div class="alert alert-success">
    <b>[5] Question:</b> 
    What does the term <i>multiplexing</i> mean? Why is this used in RAD-seq and other genomic library preparations?
</div>

In [19]:
# answer here